In [2]:
import pandas as pd
import wrds
from settings import config

# Load WRDS credentials from config
WRDS_USERNAME = config("WRDS_USERNAME")

# WRDS Connection
db = wrds.Connection(wrds_username=WRDS_USERNAME)

# Function to pull column names for a given year
def get_column_names(year):
    """Fetch column names from Markit CDS tables for a given year."""
    table_name = f"markit.cds{year}"
    query = f"SELECT * FROM {table_name} LIMIT 5"  # Fetch only a few rows to inspect structure
    df = db.raw_sql(query)
    return df.columns.tolist()

# Pull column names for two periods
years_pre_2009 = list(range(2001, 2009))   # 2001 - 2008
years_post_2009 = list(range(2009, 2025))  # 2009 - 2025

# Extract unique column names for each period
columns_pre_2009 = set()
columns_post_2009 = set()

print("Fetching column names for Markit CDS data (2001-2008)...")
for year in years_pre_2009:
    columns_pre_2009.update(get_column_names(year))
print(" Done!")

print("Fetching column names for Markit CDS data (2009-2025)...")
for year in years_post_2009:
    columns_post_2009.update(get_column_names(year))
print(" Done!")

# Convert to lists for easier viewing
columns_pre_2009 = sorted(columns_pre_2009)
columns_post_2009 = sorted(columns_post_2009)

# Display results
print("\n **Columns in Markit CDS Data (2001-2008):**")
print(columns_pre_2009)

print("\n **Columns in Markit CDS Data (2009-2025):**")
print(columns_post_2009)

# Compare Differences
added_columns = set(columns_post_2009) - set(columns_pre_2009)
removed_columns = set(columns_pre_2009) - set(columns_post_2009)

print("\n **Columns Added After 2009:**")
print(added_columns if added_columns else "None")

print("\n **Columns Removed After 2009:**")
print(removed_columns if removed_columns else "None")

# Close WRDS Connection
db.close()
print("\n Analysis Complete!")


Loading library list...
Done
Fetching column names for Markit CDS data (2001-2008)...
 Done!
Fetching column names for Markit CDS data (2009-2025)...
 Done!

 **Columns in Markit CDS Data (2001-2008):**
['_52wkhigh5y', '_52wklow5y', 'avrating', 'bankcontributionsdepthcontr', 'bankcontributionsdepthpassed', 'baspreaddepthpassed', 'batch', 'bidasktype5y', 'carriedforward', 'ccybasis', 'ccybasis10thpctlpassed', 'ccybasis90thpctlpassed', 'ccybasiscarriedforward', 'ccybasisdepthpassed', 'ccybasisstddevpassed', 'cdsassumedrecovery', 'cdsrealrecovery', 'clearingsubmissionsdepthcontr', 'clearingsubmissionsdepthpassed', 'compositecurverating', 'compositedepth5y', 'compositepricerating', 'contr10pctlpassedrecovery', 'contr10thpctlpassed', 'contr90thpctlpassed', 'contr90thpctlpassedrecovery', 'contrlqrangepassed', 'contrlqrangepassedrecovery', 'contrmaxminrangepassed', 'contrmaxminrangepassedrecovery', 'contrstddevpassedrecovery', 'convbaspread10thpctlpassed', 'convbaspread90thpctlpassed', 'convb

In [3]:
import pandas as pd
import wrds
from settings import config

# WRDS Connection
WRDS_USERNAME = config("WRDS_USERNAME")
db = wrds.Connection(wrds_username=WRDS_USERNAME)

# Define periods
years_pre_2009 = list(range(2001, 2009))   # 2001 - 2008
years_post_2009 = list(range(2009, 2025))  # 2009 - 2025

# Function to pull sample data
def get_sample_data(year, limit=5000):
    """Fetches a sample of CDS data for a given year."""
    table_name = f"markit.cds{year}"
    query = f"""
    SELECT * FROM {table_name} 
    WHERE tenor = '5Y' AND country = 'United States' 
    LIMIT {limit}
    """
    df = db.raw_sql(query)
    return df

# Fetch sample data
print("Fetching Pre-2009 Sample Data...")
df_pre_2009 = pd.concat([get_sample_data(year) for year in years_pre_2009], ignore_index=True)
print(" Done!")

print("Fetching Post-2009 Sample Data...")
df_post_2009 = pd.concat([get_sample_data(year) for year in years_post_2009], ignore_index=True)
print(" Done!")

# Function to analyze missing values
def analyze_missing_values(df, period):
    missing_values = df.isna().sum()
    total_rows = len(df)
    missing_percent = (missing_values / total_rows) * 100
    missing_summary = pd.DataFrame({'Missing Count': missing_values, 'Missing %': missing_percent})
    print(f"\n Missing Data Analysis ({period}):")
    display(missing_summary[missing_summary['Missing Count'] > 0].sort_values(by='Missing %', ascending=False))

# Compare missing values
analyze_missing_values(df_pre_2009, "2001-2008")
analyze_missing_values(df_post_2009, "2009-2025")

# Function to compute summary statistics
def compute_summary_stats(df, period):
    print(f"\n Summary Statistics for {period}:")
    display(df.describe())

# Compare summary statistics
compute_summary_stats(df_pre_2009, "2001-2008")
compute_summary_stats(df_post_2009, "2009-2025")

# Function to compare unique values across key fields
def compare_unique_values(col_name):
    unique_pre = set(df_pre_2009[col_name].dropna().unique())
    unique_post = set(df_post_2009[col_name].dropna().unique())
    added_values = unique_post - unique_pre
    removed_values = unique_pre - unique_post
    print(f"\n Unique Values Analysis for {col_name}:")
    print(f"Added after 2009: {added_values if added_values else 'None'}")
    print(f"Removed after 2009: {removed_values if removed_values else 'None'}")

# Compare unique values for key fields
compare_unique_values("ticker")
compare_unique_values("sector")
compare_unique_values("region")

# Function to compare distributions of key financial fields
def compare_distributions(col_name):
    print(f"\n Distribution Comparison for {col_name}:")
    display(pd.DataFrame({
        "Pre-2009": df_pre_2009[col_name].describe(),
        "Post-2009": df_post_2009[col_name].describe()
    }))

# Compare distributions for key financial metrics
compare_distributions("parspread")
compare_distributions("creditdv01")
compare_distributions("riskypv01")
compare_distributions("upfront")

# Close connection
db.close()
print("\n Analysis Complete!")


Loading library list...
Done
Fetching Pre-2009 Sample Data...


/var/folders/n9/vfrn1pps4m9ck4b_g2tp2bdr0000gn/T/ipykernel_5675/3965967262.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_pre_2009 = pd.concat([get_sample_data(year) for year in years_pre_2009], ignore_index=True)


 Done!
Fetching Post-2009 Sample Data...


/var/folders/n9/vfrn1pps4m9ck4b_g2tp2bdr0000gn/T/ipykernel_5675/3965967262.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_post_2009 = pd.concat([get_sample_data(year) for year in years_post_2009], ignore_index=True)


 Done!

 Missing Data Analysis (2001-2008):


...


 Distribution Comparison for riskypv01:


,Pre-2009,Post-2009
count,378.000000,19060.000000
mean,4.626614,4.711312
std,0.087970,0.192836
min,4.450000,4.050000
25%,4.560000,4.570000
50%,4.640000,4.730000
75%,4.670000,4.860000
max,4.900000,5.130000



 Distribution Comparison for upfront:


,Pre-2009,Post-2009
count,399.000000,63781.000000
mean,-0.052124,-0.080691
std,0.080214,0.105233
min,-0.208388,-0.238516
25%,-0.015494,-0.190474
50%,-0.011077,-0.034670
75%,-0.010147,-0.003312
max,0.023337,0.320047



 Analysis Complete!
